In [22]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm  # Import tqdm for progress tracking
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from datetime import timedelta

In [23]:
sales_data = pd.read_csv("../../dataset/sales.csv", parse_dates=['release_date'])

sales_data.sort_values(by='release_date', ascending=True, inplace=True)
sales_data.drop(columns=['Unnamed: 0','image_path','restock'], inplace=True)
sales_column_names = {'0':'w1_sales','1':'w2_sales','2':'w3_sales','3':'w4_sales'
                    ,'4':'w5_sales','5':'w6_sales','6':'w7_sales','7':'w8_sales'
                    ,'8':'w9_sales','9':'w10_sales','10':'w11_sales','11':'w12_sales'}
sales_data.rename(columns=sales_column_names, inplace=True)
sales_data.head(5)

,external_code,retail,season,category,color,fabric,release_date,w1_sales,w2_sales,w3_sales,w4_sales,w5_sales,w6_sales,w7_sales,w8_sales,w9_sales,w10_sales,w11_sales,w12_sales
0,5,36,SS17,long sleeve,grey,acrylic,2016-11-28,1.0,3.0,1.0,1.0,2.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0
1,2,51,SS17,long sleeve,violet,acrylic,2016-11-28,1.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0
2,5,10,SS17,long sleeve,grey,acrylic,2016-11-28,1.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
3,9,41,SS17,culottes,yellow,scuba crepe,2016-11-28,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,5,13,SS17,long sleeve,grey,acrylic,2016-11-28,1.0,2.0,4.0,0.0,3.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0


In [24]:
sales_data.shape

(106850, 19)

In [25]:
discount_data = pd.read_csv("../../dataset/price_discount_series.csv")
discount_column_names = {'0':'w1_discount','1':'w2_discount','2':'w3_discount','3':'w4_discount'
                    ,'4':'w5_discount','5':'w6_discount','6':'w7_discount','7':'w8_discount'
                    ,'8':'w9_discount','9':'w10_discount','10':'w11_discount','11':'w12_discount'}
discount_data.rename(columns=discount_column_names, inplace=True)
discount_data.head(5)

,external_code,retail,w1_discount,w2_discount,w3_discount,w4_discount,w5_discount,w6_discount,w7_discount,w8_discount,w9_discount,w10_discount,w11_discount,w12_discount,price
0,5,36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054944
1,2,51,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.054944
2,5,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054944
3,9,41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043951
4,5,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054944


In [26]:
discount_data.shape

(106850, 15)

In [27]:
sales_data = sales_data.merge(discount_data, on=["external_code","retail"])
sales_data.head(5)

,external_code,retail,season,category,color,fabric,release_date,w1_sales,w2_sales,w3_sales,...,w4_discount,w5_discount,w6_discount,w7_discount,w8_discount,w9_discount,w10_discount,w11_discount,w12_discount,price
0,5,36,SS17,long sleeve,grey,acrylic,2016-11-28,1.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054944
1,2,51,SS17,long sleeve,violet,acrylic,2016-11-28,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.054944
2,5,10,SS17,long sleeve,grey,acrylic,2016-11-28,1.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054944
3,9,41,SS17,culottes,yellow,scuba crepe,2016-11-28,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043951
4,5,13,SS17,long sleeve,grey,acrylic,2016-11-28,1.0,2.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054944


In [28]:
sales_data.shape

(106850, 32)

In [29]:
weather_data = pd.read_csv("../../dataset/vis2_weather_data.csv", parse_dates=['date'])
weather_column_names = {'locality':'w1_discount','1':'w2_discount'}
weather_data.head(5)

C:\Users\Sameera\AppData\Local\Temp\ipykernel_22740\780121790.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  weather_data = pd.read_csv("../../dataset/vis2_weather_data.csv", parse_dates=['date'])


,locality,date,avg temp °C,min temp °C,max temp °C,dew point °C,humidity %,visibility km,avg wind km/h,max wind km/h,gust km/h,slm pressure mb,avg pressure mb,rain mm
0,8,2016-01-01,6.0,2.0,11.0,2.0,69.0,10.0,9.0,13.0,0.0,1026.0,0.0,0.0
1,8,2016-02-01,9.0,2.0,14.0,5.0,69.0,10.0,11.0,17.0,0.0,1020.0,0.0,0.0
2,8,2016-03-01,12.0,8.0,15.0,10.0,86.0,9.0,12.0,22.0,52.0,1007.0,0.0,0.0
3,8,2016-04-01,11.0,6.0,14.0,8.0,85.0,9.0,10.0,17.0,0.0,1001.0,0.0,0.0
4,8,2016-05-01,13.0,10.0,16.0,11.0,86.0,9.0,10.0,19.0,0.0,1001.0,0.0,0.0


In [30]:
weather_data.shape

(89071, 14)

In [31]:
# sales_data = sales_data.merge(weather_data, left_on=['retail','release_date'], right_on=['locality','date'])
# sales_data.shape

In [32]:
gtrend_data = pd.read_csv("../../dataset/vis2_gtrends_data.csv", parse_dates=['date'], index_col=['date'])
gtrend_data.head(5)

,long sleeve,culottes,miniskirt,short sleeves,printed shirt,short cardigan,solid color top,trapeze dress,sleeveless,long cardigan,...,scottish,milano stitch,devore,hron,ottoman,fluid,flamed,fluid polyviscous,shiny jersey,goose
date,,,,,,,,,,,,,,,,,,,,,
2015-10-05,55.0,41.0,66.0,58.0,58.0,22.0,24.0,43.0,66.0,74.0,...,57,0,29,17,57,80,49,-1.0,0,34
2015-10-12,55.0,44.0,73.0,71.0,61.0,31.0,37.0,21.2,63.0,59.0,...,54,0,17,14,61,80,41,-1.0,0,39
2015-10-19,60.0,42.0,73.0,62.0,61.0,43.0,37.0,27.4,47.0,71.0,...,55,0,21,23,63,79,52,-1.0,33,41
2015-10-26,57.0,44.0,68.0,51.0,55.0,19.0,37.0,59.8,54.0,64.0,...,55,0,26,14,58,76,33,-1.0,32,37
2015-11-02,49.0,39.0,66.0,39.0,63.0,16.0,37.0,44.0,51.0,67.0,...,55,0,18,18,60,80,37,-1.0,98,38


In [33]:
def calculate_gtrend_percentageChange(row):
    release_date = row['release_date']
    trend_startDate = release_date - pd.Timedelta(days=365)
    category = row['category']
    color = row['color']
    fabric = row['fabric']

    # Select relevant trend data for the category, color, and fabric
    relvant_trends_category = gtrend_data.get(category, pd.Series(dtype='float64')).loc[trend_startDate:release_date]
    relvant_trends_color = gtrend_data.get(color, pd.Series(dtype='float64')).loc[trend_startDate:release_date]
    relvant_trends_fabric = gtrend_data.get(fabric, pd.Series(dtype='float64')).loc[trend_startDate:release_date]
    
    # Calculate the mean percentage change for each 
    category_pct_change = relvant_trends_category.pct_change().mean()
    color_pct_change = relvant_trends_color.pct_change().mean()
    fabric_pct_change = relvant_trends_fabric.pct_change().mean()

    return pd.Series({
        'category_pct_change': category_pct_change, 
        'color_pct_change': color_pct_change, 
        'fabric_pct_change': fabric_pct_change})

# Calculate percentage change of Google trend values for each row in the sales dataset
percentage_change_google_trends = sales_data.apply(lambda row: calculate_gtrend_percentageChange(row), axis=1)

# Join the new columns with the original sales data
sales_data = sales_data.join(percentage_change_google_trends)


In [34]:
sales_data.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN values (optional)
sales_data.dropna(inplace=True)

In [35]:
sales_data.to_csv('../../processedData/combinedSales.csv', index=False)
sales_data.sample(frac=0.1,random_state=0).to_csv('../../processedData/reduced_sales.csv', index=False)

In [36]:
sales_data.groupby(['retail','release_date']).nunique()

external_code  season  category  color  fabric  w1_sales  \
retail release_date                                                             
0      2016-12-05                1       1         1      1       1         1   
       2016-12-12                1       1         1      1       1         1   
       2017-01-09                1       1         1      1       1         1   
       2017-01-16                1       1         1      1       1         1   
       2017-01-30                2       1         1      2       2         1   
...                            ...     ...       ...    ...     ...       ...   
125    2019-11-25                8       1         5      5       4         4   
       2019-12-02                3       1         3      3       2         3   
       2019-12-09                1       1         1      1       1         1   
       2019-12-16                1       1         1      1       1         1   
       2019-12-30                1       1         1      1       1         1   

                     w2_sales  w3_sales  w4_sales  w5_sales  ...  w7_discount  \
retail release_date                                          ...                
0      2016-12-05           1         1         1         1  ...            1   
       2016-12-12           1         1         1         1  ...            1   
       2017-01-09           1         1         1         1  ...            1   
       2017-01-16           1         1         1         1  ...            1   
       2017-01-30           2         2         1         1  ...            1   
...                       ...       ...       ...       ...  ...          ...   
125    2019-11-25           4         5         5         4  ...            3   
       2019-12-02           2         2         2         3  ...            1   
       2019-12-09           1         1         1         1  ...            1   
       2019-12-16           1         1         1         1  ...            1   
       2019-12-30           1         1         1         1  ...            1   

                     w8_discount  w9_discount  w10_discount  w11_discount  \
retail release_date                                                         
0      2016-12-05              1            1             1             1   
       2016-12-12              1            1             1             1   
       2017-01-09              1            1             1             1   
       2017-01-16              1            1             1             1   
       2017-01-30              1            1             1             1   
...                          ...          ...           ...           ...   
125    2019-11-25              3            3             4             4   
       2019-12-02              1            1             2             2   
       2019-12-09              1            1             1             1   
       2019-12-16              1            1             1             1   
       2019-12-30              1            1             1             1   

                     w12_discount  price  category_pct_change  \
retail release_date                                             
0      2016-12-05               1      1                    1   
       2016-12-12               1      1                    1   
       2017-01-09               1      1                    1   
       2017-01-16               1      1                    1   
       2017-01-30               1      2                    1   
...                           ...    ...                  ...   
125    2019-11-25               4      5                    5   
       2019-12-02               2      3                    3   
       2019-12-09               1      1                    1   
       2019-12-16               1      1                    1   
       2019-12-30               1      1                    1   

                     color_pct_change  fabric_pct_change  
retail

In [37]:
weather_data.groupby(['locality','date']).nunique()

avg temp °C  min temp °C  max temp °C  dew point °C  \
locality date                                                              
8        2016-01-01            1            1            1             1   
         2016-01-02            1            1            1             1   
         2016-01-03            1            1            1             1   
         2016-01-04            1            1            1             1   
         2016-01-05            1            1            1             1   
...                          ...          ...          ...           ...   
242      2019-12-27            1            1            1             1   
         2019-12-28            1            1            1             1   
         2019-12-29            1            1            1             1   
         2019-12-30            1            1            1             1   
         2019-12-31            1            1            1             1   

                     humidity %  visibility km  avg wind km/h  max wind km/h  \
locality date                                                                  
8        2016-01-01           1              1              1              1   
         2016-01-02           1              1              1              1   
         2016-01-03           1              1              1              1   
         2016-01-04           1              1              1              1   
         2016-01-05           1              1              1              1   
...                         ...            ...            ...            ...   
242      2019-12-27           1              1              1              1   
         2019-12-28           1              1              1              1   
         2019-12-29           1              1              1              1   
         2019-12-30           1              1              1              1   
         2019-12-31           1              1              1              1   

                     gust km/h  slm pressure mb  avg pressure mb  rain mm  
locality date                                                              
8        2016-01-01          1                1                1        1  
         2016-01-02          1                1                1        1  
         2016-01-03          1                1                1        1  
         2016-01-04          1                1                1        1  
         2016-01-05          1                1                1        1  
...                        ...              ...              ...      ...  
242      2019-12-27          1                1                1        1  
         2019-12-28          1                1                1        1  
         2019-12-29          1                1                1        1  
         2019-12-30          1                1                1        1  
         2019-12-31          1                1                1        1  

[89071 rows x 12 columns]

# Data Preprocessing

## Checking for missing values

In [38]:
# Check for missing values
missing_values = sales_data.isnull().sum()
# Display columns with missing values
missing_values[missing_values > 0]

Series([], dtype: int64)

In [39]:
sales_data.dropna(inplace=True)

## Data Transformation

In [40]:
sales_data['release_year'] = sales_data['release_date'].dt.year
sales_data['releae_quarter'] = sales_data['release_date'].dt.quarter
sales_data['releae_month'] = sales_data['release_date'].dt.month
sales_data['releae_week'] = sales_data['release_date'].dt.week
sales_data['is_weekend'] = sales_data['releae_week'].isin([5, 6]).astype(int)

C:\Users\Sameera\AppData\Local\Temp\ipykernel_22740\4118027901.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  sales_data['releae_week'] = sales_data['release_date'].dt.week


## Outlier detection and treatment

In [41]:
from sklearn.preprocessing import OrdinalEncoder

def preprocess(df):
    # List of categorical columns to encode
    cat_cols = ['category', 'color', 'fabric', 'season']

    # List of numerical columns to check for outliers
    num_cols = ['w1_sales', 'w2_sales', 'w3_sales', 'w4_sales', 'w5_sales', 'w6_sales', 'w7_sales', 'w8_sales'
                ,'w9_sales', 'w10_sales', 'w11_sales', 'w12_sales'
                ,'w1_discount', 'w2_discount', 'w3_discount', 'w4_discount', 'w5_discount', 'w6_discount', 'w7_discount', 'w8_discount'
                ,'w9_discount', 'w10_discount', 'w11_discount', 'w12_discount']
    
    # Ordinal encoding for categorical columns
    encoder = OrdinalEncoder()
    df[cat_cols] = encoder.fit_transform(df[cat_cols])
    
    # Outlier detection for numerical columns
    for col in num_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        # Define bounds for the outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Identify the outliers
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]

        # Replace the outliers with the median of the column
        df.loc[outliers.index, col] = df[col].median()

    return df

# Apply the preprocessing function to the DataFrame
sales_data = preprocess(sales_data)

sales_data.head(5)


,external_code,retail,season,category,color,fabric,release_date,w1_sales,w2_sales,w3_sales,...,w12_discount,price,category_pct_change,color_pct_change,fabric_pct_change,release_year,releae_quarter,releae_month,releae_week,is_weekend
0,5,36,3.0,10.0,4.0,0.0,2016-11-28,1.0,3.0,1.0,...,0.0,0.054944,0.007395,0.001275,0.003783,2016,4,11,48,0
1,2,51,3.0,10.0,7.0,0.0,2016-11-28,1.0,1.0,1.0,...,0.2,0.054944,0.007395,0.000255,0.003783,2016,4,11,48,0
2,5,10,3.0,10.0,4.0,0.0,2016-11-28,1.0,3.0,1.0,...,0.0,0.054944,0.007395,0.001275,0.003783,2016,4,11,48,0
4,5,13,3.0,10.0,4.0,0.0,2016-11-28,1.0,2.0,4.0,...,0.0,0.054944,0.007395,0.001275,0.003783,2016,4,11,48,0
5,5,41,3.0,10.0,4.0,0.0,2016-11-28,2.0,1.0,2.0,...,0.0,0.054944,0.007395,0.001275,0.003783,2016,4,11,48,0


In [42]:
sales_data.shape

(83219, 40)

In [44]:
# sales_data.to_csv('../../processedData/processedSales.csv', index=False)